In [58]:
import fastf1 as f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tomli

In [59]:
with open("Data/compound_selection.toml", "rb") as toml:
    compound_selection = tomli.load(toml)

In [60]:
cache_path = os.getcwd() + "/Cache"
f.Cache.enable_cache(cache_path)

## Load Dataframes

In [77]:
def read_csv(path):
    # read csv file at path location and filter for relevant columns
    return pd.read_csv(path, 
                       header=0, 
                       true_values=["TRUE"], 
                       false_values=["FALSE"],
                       usecols=["Time", "DriverNumber", "LapTime", "LapNumber", "Stint", 
                                "PitOutTime", "PitInTime", "Compound", "TyreLife", "FreshTyre", 
                                "Team", "Driver", "TrackStatus", "IsAccurate", "RoundNumber", 
                                "EventName"]
                        )

In [79]:
def convert_timedelta(df_laps):
    df_laps[["Time", "LapTime", "PitInTime", "PitOutTime"]] = df_laps[["Time", "LapTime", "PitInTime", "PitOutTime"]].apply(pd.to_timedelta)

    return df_laps

In [80]:
def load_laps():
    df_dict = {}
    data_files = [file for file in os.listdir("data") if os.path.isfile("data/"+file)]

    for file in data_files:
        if file.startswith("all"):
            year = file.split("_")[2]
            year = int(year[:year.find(".")])
            df = read_csv("data/" + file)
            convert_timedelta(df)
            df_dict[year] = df

    return df_dict

## Data Transformation

### Add Tyre Information Columns

In [ ]:
def add_is_slick(df_laps):
    df_laps["IsSlick"] = df_laps["Compound"].apply(lambda x: x in ["SOFT", "MEDIUM", "HARD"])

    return None

FastF1 provides relative compound information (soft, medium, hard) as the `Compound` column in its `Laps` objects

The actual compound names (C1, C2, C3, C4, C5) needs to be added to maintain consistency. These will be recorded in the `CompoundName` column.

In [ ]:
def compound_relative_to_absolute(row, year):  
    compound_to_index = {"SOFT":2, "MEDIUM":1, "HARD":0}
    
    if row.loc["Compound"] not in compound_to_index:
        return row.loc["Compound"]
    else:
        try:
            if year == 2021:
                if row.loc["Compound"] in compound_to_index:
                    return 'C' + str(compound_selection_2021[row.loc["RoundNumber"]][compound_to_index[row.loc["Compound"]]])
                else: 
                    return np.nan
            elif year == 2022:
                if row.loc["Compound"] in compound_to_index:
                    return 'C' + str(compound_selection_2022[row.loc["RoundNumber"]][compound_to_index[row.loc["Compound"]]])
                else:
                    return np.nan
            else:
                raise ValueError("Year requested ({}) not available".format(year))
        except KeyError: 
            print("Update the dictionary in the cell above to reflect the compound selection in all past GP")
            
            # terminate running
            assert False

In [ ]:
df_laps_2021["CompoundName"] = df_laps_2021.apply(lambda row: compound_relative_to_absolute(row, 2021), axis=1)
df_laps_2022["CompoundName"] = df_laps_2022.apply(lambda row: compound_relative_to_absolute(row, 2022), axis=1)

### Add Timing Columns


A *representative lap time* is calculated by finding the median of the laps that meet the following condition:

- Raced on slick tyres (`IsSlick = True`) (This definition is optional and should be removed before analyzing wet races)
- `IsAccurate = True`, see definition [here](https://theoehrly.github.io/Fast-F1/core.html#fastf1.core.Laps)
- Is completed under green flag (`TrackStatus == 1`), note that this definition is stricter than the one used for `IsAccurate`

Define *valid laps* as the laps that meet all above conditions. This is recorded in the new `IsValid` column.

The fastest lap time for the session is the fastest time out of the laps where `IsPersonalBest = True` ([definition](https://theoehrly.github.io/Fast-F1/core.html#laps)). Note that this is the same definiton used by the FastF1 `pick_fastest()` method.

Using these two times as benchmarks, the following columns are added:

- `DeltaToRep`
- `DeltaToFastest`
- `PctFromRep`
- `PctFromFastest`

**Caveat**: The 2021 Turkish Grand Prix only has 1 valid lap due to it being a wet race, rendering these columns inaccurate. All other sessions have at least 600 valid laps.

In [ ]:
def check_lap_valid(row):
    return row.loc["IsSlick"] and row.loc["IsAccurate"] and row.loc["TrackStatus"] == 1

In [ ]:
df_laps_2021["IsValid"] = df_laps_2021.apply(check_lap_valid, axis=1)
df_laps_2022["IsValid"] = df_laps_2022.apply(check_lap_valid, axis=1)

In [ ]:
rep_times_2021 = {i:df_laps_2021[(df_laps_2021["RoundNumber"]==i) & (df_laps_2021["IsValid"]==True)]["LapTime"].median(numeric_only=False) for i in range(1, num_rounds_2021+1)}
rep_times_2022 = {i:df_laps_2022[(df_laps_2022["RoundNumber"]==i) & (df_laps_2022["IsValid"]==True)]["LapTime"].median(numeric_only=False) for i in range(1, num_rounds_2021+1)}

In [ ]:
fastest_times_2021 = {i:df_laps_2021[(df_laps_2021["RoundNumber"]==i) & (df_laps_2021["IsPersonalBest"] == True)]["LapTime"].min(numeric_only=False) for i in range(1, num_rounds_2021+1)}
fastest_times_2022 = {i:df_laps_2022[(df_laps_2022["RoundNumber"]==i) & (df_laps_2022["IsPersonalBest"] == True)]["LapTime"].min(numeric_only=False) for i in range(1, num_rounds_2022+1)}

In [ ]:
df_laps_2021["DeltaToRep"] = df_laps_2021.apply(lambda row: row.loc["LapTime"] - rep_times_2021[row.loc["RoundNumber"]], axis=1)
df_laps_2022["DeltaToRep"] = df_laps_2022.apply(lambda row: row.loc["LapTime"] - rep_times_2022[row.loc["RoundNumber"]], axis=1)

In [ ]:
df_laps_2021["DeltaToFastest"] = df_laps_2021.apply(lambda row: row.loc["LapTime"] - fastest_times_2021[row.loc["RoundNumber"]], axis=1)
df_laps_2022["DeltaToFastest"] = df_laps_2022.apply(lambda row: row.loc["LapTime"] - fastest_times_2022[row.loc["RoundNumber"]], axis=1)

In [ ]:
df_laps_2021["PctFromRep"] = df_laps_2021.apply(lambda row: round(row.loc["DeltaToRep"] / rep_times_2021[row.loc["RoundNumber"]] *100, 3), axis=1)
df_laps_2022["PctFromRep"] = df_laps_2022.apply(lambda row: round(row.loc["DeltaToRep"] / rep_times_2022[row.loc["RoundNumber"]] *100, 3), axis=1)

In [ ]:
df_laps_2021["PctFromFastest"] = df_laps_2021.apply(lambda row: round(row.loc["DeltaToFastest"] / fastest_times_2021[row.loc["RoundNumber"]] *100, 3), axis=1)
df_laps_2022["PctFromFastest"] = df_laps_2022.apply(lambda row: round(row.loc["DeltaToFastest"] / fastest_times_2022[row.loc["RoundNumber"]] *100, 3), axis=1)

Track evolution and diminishing fuel load has a significant influence on lap times.

The following columns are added to control for these confounding factors. Instead of comparing the lap times to a representative time for the entire event, they will be compared against a representative time at the same stage of the Grand Prix:

- `DeltaToLapRep`
- `PctFromLapRep`

The definition for the per lap representative lap times is the same as the definition for the event representative lap time.

In [ ]:
lap_reps_2021 = {}
lap_reps_2022 = {}

for round_number in pd.unique(df_laps_2021["RoundNumber"]):
    event_laps = df_laps_2021[df_laps_2021["RoundNumber"] == round_number]
    lap_numbers = pd.unique(event_laps["LapNumber"])
    event_laps = event_laps[event_laps["IsValid"]==True]
    event_lap_reps = {}
    
    for lap in lap_numbers:
        event_lap_reps[lap] = event_laps[event_laps["LapNumber"] == lap]["LapTime"].median()
    
    lap_reps_2021[round_number] = event_lap_reps
    
for round_number in pd.unique(df_laps_2022["RoundNumber"]):
    event_laps = df_laps_2022[df_laps_2022["RoundNumber"] == round_number]
    lap_numbers = pd.unique(event_laps["LapNumber"])
    event_laps = event_laps[event_laps["IsValid"]==True]    
    event_lap_reps = {}
    
    for lap in lap_numbers:
        event_lap_reps[lap] = event_laps[event_laps["LapNumber"] == lap]["LapTime"].median()
    
    lap_reps_2022[round_number] = event_lap_reps

In [ ]:
df_laps_2021["DeltaToLapRep"] = df_laps_2021.apply(lambda row: row.loc["LapTime"] - lap_reps_2021[row.loc["RoundNumber"]][row.loc["LapNumber"]], axis=1)
df_laps_2022["DeltaToLapRep"] = df_laps_2022.apply(lambda row: row.loc["LapTime"] - lap_reps_2022[row.loc["RoundNumber"]][row.loc["LapNumber"]], axis=1)

In [ ]:
df_laps_2021["PctFromLapRep"] = df_laps_2021.apply(lambda row: round(row.loc["DeltaToLapRep"] / lap_reps_2021[row.loc["RoundNumber"]][row.loc["LapNumber"]] * 100, 3), axis=1)
df_laps_2022["PctFromLapRep"] = df_laps_2022.apply(lambda row: round(row.loc["DeltaToLapRep"] / lap_reps_2022[row.loc["RoundNumber"]][row.loc["LapNumber"]] * 100, 3), axis=1)

### Add Second Columns

Timedelta objects are hard to make visualizations with.  New columns that store same information in seconds will be created to address this issue. These new columns are denoted with a lower case s at the beginning.

- `sLapTime`
- `sDeltaToRep`
- `sDeltaToFastest`
- `sDeltaToLapRep`

In [ ]:
df_laps_2021["sLapTime"] = df_laps_2021.apply(lambda row: row.loc["LapTime"].total_seconds(), axis=1)
df_laps_2022["sLapTime"] = df_laps_2022.apply(lambda row: row.loc["LapTime"].total_seconds(), axis=1)

In [ ]:
df_laps_2021["sDeltaToRep"] = df_laps_2021.apply(lambda row: row.loc["DeltaToRep"].total_seconds(), axis=1)
df_laps_2022["sDeltaToRep"] = df_laps_2022.apply(lambda row: row.loc["DeltaToRep"].total_seconds(), axis=1)

In [ ]:
df_laps_2021["sDeltaToFastest"] = df_laps_2021.apply(lambda row: row.loc["DeltaToFastest"].total_seconds(), axis=1)
df_laps_2022["sDeltaToFastest"] = df_laps_2022.apply(lambda row: row.loc["DeltaToFastest"].total_seconds(), axis=1)

In [ ]:
df_laps_2021["sDeltaToLapRep"] = df_laps_2021.apply(lambda row: row.loc["DeltaToLapRep"].total_seconds(), axis=1)
df_laps_2022["sDeltaToLapRep"] = df_laps_2022.apply(lambda row: row.loc["DeltaToLapRep"].total_seconds(), axis=1)

## Export To CSV

In [ ]:
df_laps_2021.to_csv("transformed_laps_2021.csv")
df_laps_2022.to_csv("transformed_laps_2022.csv")